<a href="https://colab.research.google.com/github/SVGS-EVA4/Phase2/blob/master/S4-FaceRecognitionPart2/Session4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install torch==1.5.0 torchvision==0.6.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 603.7MB 29kB/s 
     |████████████████████████████████| 6.5MB 5.8MB/s 
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
  Found existing installation: torchvision 0.7.0+cu101
    Uninstalling torchvision-0.7.0+cu101:
      Successfully uninstalled torchvision-0.7.0+cu101


In [2]:
!git clone https://github.com/Sushmitha-Katti/PyTNet
import sys
sys.path.append("PyTNet")

Cloning into 'PyTNet'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 160 (delta 86), reused 99 (delta 40), pack-reused 0
Receiving objects: 100% (160/160), 42.16 KiB | 8.43 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [86]:
from torch import nn, optim, as_tensor
import torchvision
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
from models.inception_resnet_v1 import InceptionResnetV1
import cv2
from PIL import Image
from pdb import set_trace
import time
import copy
from pathlib import Path
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage import io, transform
from tqdm import trange, tqdm
import csv
import glob
import dlib
import pandas as pd
import numpy as np
from torchsummary import summary

In [87]:
# Total No Of Classes
!ls 'drive/My Drive/lfw_funneled'| wc -l

5770


In [88]:
train_transforms =transforms.Compose([
        transforms.Resize(size=(224, 224),interpolation=2),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])   # Normalized value 
    ])

dir = '/content/drive/My Drive/lfw_funneled'
dataset = torchvision.datasets.ImageFolder(dir,transform = train_transforms)

In [89]:
def load(trainset,testset,seed=1,batch_size=128,num_workers=4,pin_memory=True):
	

	#Get the Train and Test Set
	# trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
	# testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)


	SEED = 1

	# CUDA?
	cuda = torch.cuda.is_available()


	# For reproducibility
	torch.manual_seed(SEED)

	if cuda:
			torch.cuda.manual_seed(SEED)

	# dataloader arguments - something you'll fetch these from cmdprmt
	dataloader_args = dict( batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory) if cuda else dict(shuffle=True, batch_size=64)

	trainloader = torch.utils.data.DataLoader(trainset, **dataloader_args)
	testloader = torch.utils.data.DataLoader(testset, **dataloader_args)


	return  trainloader, testloader

In [105]:
from torch.utils.data import Dataset, random_split
train_len = len(dataset)*70//100
test_len = len(dataset) - train_len 
train_set, val_set = random_split(dataset, [train_len, test_len])


In [106]:
import train_test_loader
trainloader, testloader = load(train_set,val_set,batch_size=32)
num_classes = len(dataset.classes)

In [128]:
device = "cuda"
model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes = num_classes).to(device)
model.logits.out_features = num_classes


In [129]:

total_child = len(list(model.children()))
ct = 0
print("\nFreezed layer except below childs/layers \n")

for child in model.children():
  ct += 1
  if ct < total_child - 5:
    for param in child.parameters():
        param.requires_grad = False
  else:
    print(child)


Freezed layer except below childs/layers 

Block8(
  (branch0): BasicConv2d(
    (conv): Conv2d(1792, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (branch1): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(1792, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (1): BasicConv2d(
      (conv): Conv2d(192, 192, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (2): BasicConv2d(
      (conv): Conv2d(192, 192, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
  )
  (conv2d

In [130]:
for param in model.conv2d_3b.parameters():
  a = 0 #do nothing
print("requires_grad value set to early layer:",param.requires_grad)

requires_grad value set to early layer: False


In [131]:
import torch

#Training & Testing Loops
from tqdm.notebook import tqdm
from tqdm import tqdm_notebook

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, criterion, epoch,scheduler = False):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    #loss = F.nll_loss(y_pred, target)
    loss = criterion(y_pred, target)
    

    # Backpropagation
    loss.backward()
    optimizer.step()
    if(scheduler):
      scheduler.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(f'Loss={loss.item():0.2f} Acc={100*correct/processed:0.2f}')
  train_losses.append(loss)
  train_acc.append(100*correct/processed)
  return train_losses,train_acc


def test(model, device, criterion, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
	
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))
    return test_losses,test_acc



test_loss = []
test_acc = []
train_loss = []
train_loss = []
LR = []
train_scheduler = False

def train_model(model,device,trainloader,testloader,optimizer,criterion,EPOCHS,scheduler = False,batch_scheduler = False ,best_acc = 0,path = "/content/gdrive/My Drive/API/bestmodel.pt"):
  for epoch in range(EPOCHS):
      print("EPOCH:", epoch+1,'LR:',optimizer.param_groups[0]['lr'])
      LR.append(optimizer.param_groups[0]['lr'])
      train_scheduler = False

      if(batch_scheduler):
        train_scheduler = scheduler
      train_loss, train_acc = train(model, device, trainloader, optimizer, criterion, epoch,train_scheduler)
      
      test_loss , test_acc = test(model, device, criterion, testloader)
      if(scheduler and not batch_scheduler and not isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau)): 
        scheduler.step()

      elif(scheduler and not batch_scheduler and  isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau)):
        scheduler.step(test_loss[-1])
      
      
      if(test_acc[-1]>best_acc):
        print("accuracy increased, Saving model....")
        best_acc = test_acc[-1]
        torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': test_loss[-1],
              }, path)


In [132]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR,StepLR,MultiStepLR, ReduceLROnPlateau

optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9, nesterov=True) 
criterion = nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

In [134]:
train_model(model,device,trainloader,testloader,optimizer,criterion,10,scheduler,batch_scheduler=False,best_acc = 0,path = "newtinymodel.pt")

EPOCH: 1 LR: 0.01




Test set: Average loss: 0.1721, Accuracy: 1364/3999 (34.11%)

accuracy increased, Saving model....
EPOCH: 2 LR: 0.01




Test set: Average loss: 0.1572, Accuracy: 1634/3999 (40.86%)

accuracy increased, Saving model....
EPOCH: 3 LR: 0.01




Test set: Average loss: 0.1491, Accuracy: 1813/3999 (45.34%)

accuracy increased, Saving model....
EPOCH: 4 LR: 0.01




Test set: Average loss: 0.1464, Accuracy: 1927/3999 (48.19%)

accuracy increased, Saving model....
EPOCH: 5 LR: 0.01


...



Test set: Average loss: 0.1475, Accuracy: 2033/3999 (50.84%)

accuracy increased, Saving model....
EPOCH: 7 LR: 0.01




Test set: Average loss: 0.1482, Accuracy: 2072/3999 (51.81%)

accuracy increased, Saving model....
EPOCH: 8 LR: 0.01




Test set: Average loss: 0.1482, Accuracy: 2084/3999 (52.11%)

accuracy increased, Saving model....
EPOCH: 9 LR: 0.01




Test set: Average loss: 0.1494, Accuracy: 2092/3999 (52.31%)

accuracy increased, Saving model....
EPOCH: 10 LR: 0.01




Test set: Average loss: 0.1503, Accuracy: 2098/3999 (52.46%)

accuracy increased, Saving model....


In [123]:
torch.save(model, '/content/drive/My Drive/EV4P2/session4/lfw_model.pt')

In [125]:
traced_model = torch.jit.trace(model.to('cpu'), torch.randn(1, 3, 224, 224))

In [126]:
traced_model.save('/content/drive/My Drive/EV4P2/session4/lfw_plus_model.traced.pt')